In [1]:
import numpy as np
from scipy import optimize

 - https://stackoverflow.com/questions/42388139/how-to-compute-standard-deviation-errors-with-scipy-optimize-least-squares
 - https://stackoverflow.com/questions/14854339/in-scipy-how-and-why-does-curve-fit-calculate-the-covariance-of-the-parameter-es
 - https://stats.stackexchange.com/questions/38115/how-do-i-interpret-the-covariance-matrix-from-a-curve-fit
 - https://stackoverflow.com/questions/70754455/how-to-get-hessian-matrix-from-python-minimize-function

In [2]:
def model(x, a, b, c):
    return a * x[:, 0]**2 + b * x[:, 0] + c

In [3]:
np.random.seed(12345)
X = np.linspace(-1, 1, 30).reshape(-1, 1)
y = model(X, 3, 2, 1)
s = 0.1 * np.ones(y.size)
n = s * np.random.normal(size=y.size)
yn = y + n

In [4]:
popt, pcov = optimize.curve_fit(model, X, yn, sigma=s, absolute_sigma=True)
popt, pcov

(array([3.00876769, 2.00038216, 1.03068484]),
 array([[ 3.29272571e-03, -1.11752553e-11, -1.17327007e-03],
        [-1.11752553e-11,  9.35483883e-04,  2.98424026e-12],
        [-1.17327007e-03,  2.98424026e-12,  7.51395072e-04]]))

In [5]:
def loss_factory(x, y, sigma=1.):
    def wrapped(beta):
        return 0.5 * np.sum(np.power((y - model(x, *beta)) / sigma, 2))
    return wrapped

In [6]:
loss = loss_factory(X, yn, sigma=s)

In [7]:
sol1 = optimize.minimize(loss, x0=[1, 1, 1])
sol1

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 16.31766343837042
        x: [ 3.009e+00  2.000e+00  1.031e+00]
      nit: 5
      jac: [ 1.907e-06  1.192e-06 -4.768e-07]
 hess_inv: [[ 3.293e-03  1.104e-11 -1.173e-03]
            [ 1.104e-11  9.355e-04 -1.734e-12]
            [-1.173e-03 -1.734e-12  7.514e-04]]
     nfev: 32
     njev: 8

In [8]:
sol2 = optimize.minimize(loss, x0=[1, 1, 1], method="L-BFGS-B")
sol2

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 16.317663438454282
        x: [ 3.009e+00  2.000e+00  1.031e+00]
      nit: 8
      jac: [-5.720e-05  2.917e-04 -4.182e-04]
     nfev: 36
     njev: 9
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

In [9]:
sol2.hess_inv.todense()

array([[ 0.03719202,  0.02371079, -0.02612526],
       [ 0.02371079,  0.02215422, -0.01929008],
       [-0.02612526, -0.01929008,  0.01984616]])

In [22]:
sol3 = optimize.minimize(loss, x0=[1, 1, 1], method="BFGS")
sol3

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 16.31766343837042
        x: [ 3.009e+00  2.000e+00  1.031e+00]
      nit: 5
      jac: [ 1.907e-06  1.192e-06 -4.768e-07]
 hess_inv: [[ 3.293e-03  1.104e-11 -1.173e-03]
            [ 1.104e-11  9.355e-04 -1.734e-12]
            [-1.173e-03 -1.734e-12  7.514e-04]]
     nfev: 32
     njev: 8

In [10]:
def residuals_factory(x, y, sigma=1.):
    def wrapped(beta):
        return (y - model(x, *beta)) / sigma
    return wrapped

In [11]:
residuals = residuals_factory(X, yn, sigma=s)

In [12]:
sol0 = optimize.least_squares(residuals, x0=[1, 1, 1])
sol0

     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [-5.954e-01  9.965e-02 ... -3.855e-01  9.455e-01]
           x: [ 3.009e+00  2.000e+00  1.031e+00]
        cost: 16.317663438370303
         jac: [[-1.000e+01  1.000e+01 -1.000e+01]
               [-8.668e+00  9.310e+00 -1.000e+01]
               ...
               [-8.668e+00 -9.310e+00 -1.000e+01]
               [-1.000e+01 -1.000e+01 -1.000e+01]]
        grad: [ 1.070e-07 -1.632e-06  1.722e-07]
  optimality: 1.632403261453419e-06
 active_mask: [ 0.000e+00  0.000e+00  0.000e+00]
        nfev: 4
        njev: 3

In [13]:
U, s, Vh = np.linalg.svd(sol0.jac, full_matrices=False)
tol = np.finfo(float).eps*s[0]*max(sol0.jac.shape)
w = s > tol
cov = (Vh[w].T/s[w]**2) @ Vh[w]

In [14]:
cov

array([[ 3.29272571e-03,  5.77282993e-12, -1.17327008e-03],
       [ 5.77282993e-12,  9.35483875e-04,  1.14915683e-12],
       [-1.17327008e-03,  1.14915683e-12,  7.51395089e-04]])

In [15]:
pcov

array([[ 3.29272571e-03, -1.11752553e-11, -1.17327007e-03],
       [-1.11752553e-11,  9.35483883e-04,  2.98424026e-12],
       [-1.17327007e-03,  2.98424026e-12,  7.51395072e-04]])